# Creating Synthetic Vertebra CT Scans from Segmentation Maps

1) Install the needed libraries

In [2]:
!pip install nibabel

     ---------------------------------------- 3.3/3.3 MB 114.7 kB/s eta 0:00:00


In [140]:
!pip install nPerlinNoise

In [58]:
!pip install mrivis

2) Write the needed opencv helper functions. These functions do a lot of useful things like find the contours, errode and dilate, find boundaries of contours, generate and apply noise, etc

In [87]:
import cv2
import nibabel as nib
import numpy as np
from perlin_noise import PerlinNoise
from NPerlinNoise import *
from mrivis import SlicePicker
import matplotlib.pyplot as plt


def contourIsWide(contour):
    _,_,w,h = cv2.boundingRect(contour)
    if w/h > 1:
        return True
    return False

def keepContoursOnSameLineHavingLargestArea(contours):
    groups = getContourGroups(contours)
    contours_to_keep = []
    for group in groups:
        group_contours = group['contours']
        
        areas = []
        
        for contour in group_contours:
            areas.append(cv2.contourArea(contour))
        
        best_idx = areas.index(max(areas))
        
        contours_to_keep.append(group_contours[best_idx])
        
    return contours_to_keep

def seperateContoursVerticallyAndKeepLargestArea(contours):
    cxs = []
    for contour in contours:
        cx,cy = getControurCentroid(contour)
        cxs.append(cx)
    average_cx = sum(cxs)/len(cxs)
    
    left_group = []
    right_group = []
    
    for contour in contours:
        cx,cy = getControurCentroid(contour)
        if cx<average_cx:
            left_group.append(contour)
        else:
            right_group.append(contour)
    
    left_area = sum([cv2.contourArea(contour) for contour in left_group])
    right_area = sum([cv2.contourArea(contour) for contour in right_group])
    
    if left_area>right_area:
        return left_group
    return right_group

def getBestSlice(data, dim = 1):
    slices = SlicePicker(data, dim)
    slices_list = [s[1] for s in list(slices)]
    try:
        return slices_list[len(slices_list)//2]
    except:
        return None

def getBestContours(image):
    kernel_sizes = [0, 5, 8, 10]
    num_contours_for_kernels = [0]*len(kernel_sizes)
    
    for i in range(len(kernel_sizes)):
        _, thresh = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        kernel = np.ones((kernel_sizes[i], kernel_sizes[i]), np.uint8)
        opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
        contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        num_contours_for_kernels[i] = len(contours)
       
    best_kernel_idx =  num_contours_for_kernels.index(max(num_contours_for_kernels))
    
    _, thresh = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((kernel_sizes[best_kernel_idx], kernel_sizes[best_kernel_idx]), np.uint8)
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours

##apply the sigmoid function with a translation and power. Useful to create a step function that zeros out all values below a threshold.
## used to create the boundary of the contours.
def customSigmoid(matrix, power, translation):
    return 1/(1 + np.exp(-(matrix-translation))**power)
## define a random perlin noise
def getNoise():
    return Noise(
        seed=None,
        frequency=8,
        waveLength=64,
        warp=None,
        _range=None,
        octaves=8,
        persistence=0.5,
        lacunarity=2
    )

def getControurCentroid(contour):
    M = cv2.moments(contour)
    if M['m00']:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
    else:
        cx, cy = (0,0)
    
    return cx,cy

def getRelativeContourCentroid(contour):
    x, y, w, h = cv2.boundingRect(contour)
    
    # Calculate the centroid of the contour
    M = cv2.moments(contour)
    if M['m00']:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
    else:
        cx, cy = (0,0)
    
    # Calculate the centroid relative to the top-left edge of the contour
    rel_cx = cx - x
    rel_cy = cy - y
    
    return rel_cx, rel_cy

## this follows a huristic to keep only contours that have no neighbors to their left or right. Makes identifying vertebra easier when looking at the front CT slice.
def eliminateContoursOnSameLine(contours):
    contours_not_on_same_line = []
    
    contour_centroids = [getControurCentroid(contour) for contour in contours]
    for i in range(len(contours)):
        add_contour = True
        for j in range(len(contours)):
            if i!=j and abs(contour_centroids[i][1] - contour_centroids[j][1]) < 10:
                add_contour = False
                break
        if add_contour:
            contours_not_on_same_line.append(contours[i])
    
    contours_not_having_gaps = []
    
    contour_centroids = [getControurCentroid(contour) for contour in contours_not_on_same_line]
    
    for i in range(len(contours_not_on_same_line)):
        add_contour = True
        if i>0 and abs(contour_centroids[i][1] - contour_centroids[i-1][1]) > 70:
            add_contour = False
        if i<len(contours_not_on_same_line)-1 and abs(contour_centroids[i][1] - contour_centroids[i+1][1]) > 70:
            add_contour = False
            contours_not_having_gaps = []
        if add_contour:
            contours_not_having_gaps.append(contours_not_on_same_line[i])
    
    return contours_not_having_gaps


## what constitutes a contour group is a group of bones that have close centroids on the y-axis
def getContourGroups(contours):
    contour_groups = []
    for contour in contours:
        cx, cy = getControurCentroid(contour)
        # Check if the contour belongs to an existing group
        group_found = False
        for group in contour_groups:
            meanCentroid = np.array(group['centroids']).mean(0)
            ##this used to be 20 but I changed it later!!
            if abs(meanCentroid[1] - cy) < 25:
                group['contours'].append(contour)
                group['centroids'].append((cx, cy))
                group_found = True
                break

        # If the contour doesn't belong to an existing group, create a new group
        if not group_found:
            contour_groups.append({'contours': [contour], 'centroids': [(cx, cy)]})
            
    return contour_groups

#draws the border of the conrour group and applies the noise h
def drawContourGroupCT(group, mask, h):
    # Create a filled contour
    cv2.drawContours(mask, [group], 0, 255, -2)

    # Create a distance transform
    dist_transform = cv2.distanceTransform(mask, cv2.DIST_L2, 5)

    # Normalize the distance transform
    norm_dist_transform = cv2.normalize(dist_transform, None, 0, 1.0, cv2.NORM_MINMAX)
    
    # Generate a gradient using the normalized distance transform
    gradient = customSigmoid(1 - norm_dist_transform, 20, 0.8) * 255
    gradient += customSigmoid(norm_dist_transform, 30, 0.9) * 255 * h
    gradient = np.minimum(gradient, 255)
    gradient = gradient.astype(np.uint8)

    rows, cols = gradient.shape

#     rand = np.random.random(image.shape)*50
#     gradient = np.minimum(np.maximum((gradient + rand),0),255)
    gradient = np.minimum(np.maximum((gradient + h*255),0),255)

#     for i in range(rows):
#         for j in range(cols):
#             noise_val = h[i, j]
#             gradient[i, j] = max(0, min(255, int(noise_val*gradient[i, j])))

    # Apply the gradient to the contour region
    mask[mask == 255] = gradient[mask == 255]

#this is just for demonstration purpuses. same as above basically
def drawContourGroupCTShowSteps(group, image, h):
    # Create a filled contour
    mask0 = np.zeros_like(image)

    res1, res2 = mask0.shape
    mask1 = np.zeros_like(image)

    res1, res2 = mask1.shape
    
    mask2 = np.zeros_like(image)

    res1, res2 = mask1.shape
    
    cv2.drawContours(mask1, [group], 0, 255, -2)

    # Create a distance transform
    dist_transform = cv2.distanceTransform(mask1, cv2.DIST_L2, 5)
#     dist_transform = cv2.distanceTransform(mask2, cv2.DIST_L2, 5)

    # Normalize the distance transform
    norm_dist_transform = cv2.normalize(dist_transform, None, 0, 1.0, cv2.NORM_MINMAX)
    contrast_factor = np.random.uniform(0.2, 1)  # Adjust the range as needed

    h = cv2.convertScaleAbs(h*255, alpha=contrast_factor, beta=1)/255
    # Generate a gradient using the normalized distance transform
    gradient = customSigmoid(1 - norm_dist_transform, 20, 0.8) * 255
    mask1[mask1 == 255] = gradient[mask1 == 255]
    gradient += customSigmoid(norm_dist_transform, 30, 0.9) * 255 * h
    gradient = np.minimum(gradient, 255)
    gradient = gradient.astype(np.uint8)

    rows, cols = gradient.shape

#     rand = np.random.random(image.shape)*50
#     gradient = np.minimum(np.maximum((gradient + rand),0),255)
    gradient = np.minimum(np.maximum((gradient + h*255),0),255)

#     for i in range(rows):
#         for j in range(cols):
#             noise_val = h[i, j]
#             gradient[i, j] = max(0, min(255, int(noise_val*gradient[i, j])))

    # Apply the gradient to the contour region
    cv2.drawContours(mask2, [group], 0, 255, -2)
    mask2[mask2 == 255] = gradient[mask2 == 255]
    
    x, y, w, h = cv2.boundingRect(group)
    offset = 100
    cv2.drawContours(mask0, [group], 0, 255, -2)
    mask0 = mask0[y-offset:y+h+offset, x-offset:x+w+offset]
    mask1 = mask1[y-offset:y+h+offset, x-offset:x+w+offset]
    mask2 = mask2[y-offset:y+h+offset, x-offset:x+w+offset]
    plt.imshow(mask0, cmap='gray')
    plt.show()
    plt.imshow(mask1, cmap='gray')
    plt.show()
    plt.imshow(mask2, cmap='gray')
    plt.show()

    

3) Use the helper functions to create the transformation needed and save the images that were produced from the original CT Scan file

In [2]:
def transformSegmentationToCTs(image, save_folder, prefix):
    noise = getNoise()

    _, thresh = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contour_groups = getContourGroups(contours)
    for i in range(len(contour_groups)):

        extendedContours = []
        #Commented loops below if generating extended vertebra CTs 
#         if i == 0 or i == len(contour_groups) -1:
#             for contour in contour_groups[i]['contours']:
#                 _,_,w,height = cv2.boundingRect(contour)
#                 if height>20:
#                     extendedContours.append(contour)
#         else:
#             for j in range(i-1,i+2):
#                 for contour in contour_groups[j]['contours']:
#                     extendedContours.append(contour)

        #loop below if generating single vertebra CTs
        for contour in contour_groups[i]['contours']:
                _,_,w,height = cv2.boundingRect(contour)
                if height>20:
                    extendedContours.append(contour)
                    
        if len(extendedContours) == 0:
            continue
        
        mask = np.zeros_like(image)

        res1, res2 = mask.shape

        gradients = Gradient.scope()
        h, coordsMesh = perlinGenerator(noise,(0, res1),(0, res2))
        
        h = h[:res1,:res2]
        
        # Change noise contrast
        contrast_factor = np.random.uniform(0.2, 1)  # Adjust the range as needed

        h = cv2.convertScaleAbs(h*255, alpha=contrast_factor, beta=1)/255

        for cnt in extendedContours:
            drawContourGroupCT(cnt, mask, h)

        first_cx, first_cy = getRelativeContourCentroid(extendedContours[0])

        last_cx, last_cy = getRelativeContourCentroid(extendedContours[-1])

        all_contours = np.concatenate(extendedContours)

        x, y, w, h = cv2.boundingRect(all_contours)
        cropped_image = None
        #Commented crop below if generating extended vertebra CTs
#         if i == 0 or i == len(contour_groups) -1:
#             cropped_image = mask[y:y+h, x:x+w]
#         else:
#             cropped_image = mask[y+last_cy:y+h-first_cy, x:x+w]

        #Crop below if for generating single vertebra CTs
        cropped_image = mask[y:y+h, x:x+w]
        try:
            cv2.imwrite(save_folder + '/' + prefix+'_'+str(i) +".png", cropped_image)
        except:
            pass

4) loop over all nii files from the CTSpine1K dataset, get the best front facing slice index, and then get the best contours from this slice and apply the transformation.

In [8]:
import os
import nibabel as nib
import numpy as np
from tqdm import tqdm
directory_name = 'data'
directory = os.fsencode(directory_name)
save_directory_name = 'seg2ct3'    

for file in tqdm(os.listdir(directory)):
    filename = os.fsdecode(file)
    path = directory_name+'/'+filename
    # Load NIfTI file
    nii_file = nib.load(path)
    # Get the data array
    data = nii_file.get_fdata()
#     slice_index = 256  # Change this to the desired slice index
#     image = data[:, slice_index, ::-1].T.astype(np.uint8)*255
#     if image.max() == 0:
    slice_index = getBestSlice(data)
    if not slice_index:
        continue
    image = data[:, slice_index, ::-1].T.astype(np.uint8)*255
    best_contours =  getBestContours(image)
    wanted_contours = eliminateContoursOnSameLine(best_contours)
    image = np.zeros_like(image)
    for contour in wanted_contours:
        cv2.drawContours(image, [contour], 0, 255, -2)
#     plt.imshow(image)
#     plt.show()
    transformSegmentationToCTs(image, save_directory_name, filename)


100%|██████████████████████████████████████████████████████████████████████████████| 1005/1005 [34:56<00:00,  2.09s/it]


5) same as above except it looks at the best side facing slice, gets the contours, and applies the transformation.

In [9]:
import os
import nibabel as nib
import numpy as np
from tqdm import tqdm


directory_name = 'data'
directory = os.fsencode(directory_name)
save_directory_name = 'seg2ct3/side'    

for file in tqdm(os.listdir(directory)):
    filename = os.fsdecode(file)
    path = directory_name+'/'+filename
    # Load NIfTI file
    nii_file = nib.load(path)
    # Get the data array
    data = nii_file.get_fdata()
    slice_index = getBestSlice(data, 0)
    if not slice_index:
        continue
    image = data[slice_index, :, ::-1].T.astype(np.uint8)*255
    best_contours =  getBestContours(image)
    wanted_contours = seperateContoursVerticallyAndKeepLargestArea(best_contours)
    wanted_contours = keepContoursOnSameLineHavingLargestArea(wanted_contours)
    image = np.zeros_like(image)
    for contour in wanted_contours:
        cv2.drawContours(image, [contour], 0, 255, -2)
#     plt.imshow(image)
#     plt.show()
    transformSegmentationToCTs(image, save_directory_name, filename)

100%|████████████████████████████████████████████████████████████████████████████| 1005/1005 [1:44:46<00:00,  6.26s/it]


6) apply padding to all the generated images so that they are centered in a 128x128 pixel image. We first apply some checks such as trying to fit a polygon and counting the verticies, and checking the width to height ratio. This is done to eliminate some weird images that we couldn't eliminate with the huristics we applied earier.

In [47]:
from tqdm import tqdm
import os
directory_name = 'seg2ct3'
directory = os.fsencode(directory_name)
save_directory_name = 'padded_seg2ct3'    

for file in tqdm(os.listdir(directory)):
    try:
        filename = os.fsdecode(file)
        path = directory_name+'/'+filename
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        
        _, thresh = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours)>1:
            continue
        
        for cnt in contours:
            epsilon = 5
            approx = cv2.approxPolyDP(cnt, epsilon, True)
        
        if len(approx)>6:
            continue

        h,w = img.shape
        
        if h/w>1:
            continue

        padded_img = np.zeros((128,128))
        h1,w1 = padded_img.shape
    
        padded_img[(h1-h)//2:(h1+h)//2, (w1-w)//2:(w1+w)//2] = img
    except:
        continue

    try:
        cv2.imwrite(save_directory_name + '/' + filename, padded_img)
    except:
        pass
    # Load NIfTI file

100%|████████████████████████████████████████████████████████████████████████████| 3262/3262 [00:01<00:00, 2009.83it/s]


In [48]:
from tqdm import tqdm
import os
directory_name = 'seg2ct3/side'
directory = os.fsencode(directory_name)
save_directory_name = 'padded_seg2ct3'    

for file in tqdm(os.listdir(directory)):
    try:
        filename = os.fsdecode(file)
        path = directory_name+'/'+filename
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

        h,w = img.shape
        
        _, thresh = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours)>1:
            continue
        
        for cnt in contours:
            epsilon = 5
            approx = cv2.approxPolyDP(cnt, epsilon, True)
        
        if len(approx)>6:
            continue

        h,w = img.shape
        
        if h/w>1:
            continue

        padded_img = np.zeros((128,128))
        h1,w1 = padded_img.shape
    
        padded_img[(h1-h)//2:(h1+h)//2, (w1-w)//2:(w1+w)//2] = img
    except:
        continue

    try:
        cv2.imwrite(save_directory_name + '/side_' + filename, padded_img)
    except:
        pass
    # Load NIfTI file

100%|████████████████████████████████████████████████████████████████████████████| 6726/6726 [00:03<00:00, 1874.12it/s]


7) (Unrelated) Here we take the AUBMC 256x256 pixel images, and crop them down to 128x128 pixels. This is done just so that the AUBMC images are the same size as the images we generated, and to reduce the amount of data we need to generate by the GAN.

In [64]:
from tqdm import tqdm
import os
directory_name = 'cropped_good'
directory = os.fsencode(directory_name)
save_directory_name = 'cropped_healthy_vertebra'    

for file in tqdm(os.listdir(directory)):
    filename = os.fsdecode(file)
    path = directory_name+'/'+filename
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    width, height = img.shape[1], img.shape[0]

    mid_x, mid_y = int(width/2), int(height/2)
    crop_offset = int(128//2)
    crop_img = img[mid_y-crop_offset:mid_y+crop_offset, mid_x-crop_offset:mid_x+crop_offset]
    try:
        cv2.imwrite(save_directory_name + '/' + filename, crop_img)
    except:
        pass
   

100%|███████████████████████████████████████████████████████████████████████████████| 377/377 [00:00<00:00, 752.55it/s]


In [65]:
from tqdm import tqdm
import os
directory_name = 'cropped_bad'
directory = os.fsencode(directory_name)
save_directory_name = 'cropped_fractured_vertebra'    

for file in tqdm(os.listdir(directory)):
    filename = os.fsdecode(file)
    path = directory_name+'/'+filename
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    width, height = img.shape[1], img.shape[0]

    mid_x, mid_y = int(width/2), int(height/2)
    crop_offset = int(128//2)
    crop_img = img[mid_y-crop_offset:mid_y+crop_offset, mid_x-crop_offset:mid_x+crop_offset]
    try:
        cv2.imwrite(save_directory_name + '/' + filename, crop_img)
    except:
        pass
   

100%|███████████████████████████████████████████████████████████████████████████████| 296/296 [00:00<00:00, 663.97it/s]
